In [ ]:
import tkinter as tk
from tkinter import simpledialog, END
import socket
import threading
import logging


class ChatClient:
    def __init__(self, master):
        self.master = master
        self.master.title("Chat Client")

        self.username = self.get_username()
        self.create_widgets()

        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect(('localhost', 1234))

        self.client_socket.send(f"{self.username}".encode())

        self.log_filename = f"{self.username}_client_log.txt"
        logging.basicConfig(filename=self.log_filename, level=logging.INFO)

        receive_thread = threading.Thread(target=self.receive_messages)
        receive_thread.start()

    def get_username(self):
        username = simpledialog.askstring("Username", "Enter your username:")
        return username

    def create_widgets(self):
        self.chat_history = tk.Text(self.master, wrap=tk.WORD, font=("Helvetica", 18), state=tk.DISABLED)
        self.chat_history.pack(expand=True, fill="both")

        self.message_entry = tk.Entry(self.master, font=("Helvetica", 14))
        self.message_entry.pack(side=tk.LEFT, expand=True, fill="both")

        self.send_button = tk.Button(self.master, text="Send", command=self.send_message, font=("Helvetica", 14))
        self.send_button.pack(side=tk.RIGHT)

    def receive_messages(self):
        while True:
            try:
                message = self.client_socket.recv(1024).decode()
                if message:
                    self.display_message(message)
                    self.log_message(f"Received: {message}")
            except ConnectionAbortedError:
                break
            except Exception as e:
                print(f"Error receiving message: {e}")
                break

    def send_message(self):
        message = self.message_entry.get()
        if message:
            self.client_socket.send(message.encode())
            self.display_message(f"{self.username}: {message}")
            self.log_message(f"Sent: {message}")
            self.message_entry.delete(0, tk.END)

    def display_message(self, message):
        self.chat_history.config(state=tk.NORMAL)

        tag_name = f"{self.username}"
        self.chat_history.tag_configure(tag_name)

        self.chat_history.insert(tk.END, message + "\n", tag_name)

        self.chat_history.config(state=tk.DISABLED)
        self.chat_history.yview(tk.END)

    def log_message(self, message):
        logging.info(message)

    def on_closing(self):
        self.client_socket.close()
        self.master.destroy()


if __name__ == "__main__":
    root = tk.Tk()
    root.geometry("800x600")
    client = ChatClient(root)
    root.protocol("WM_DELETE_WINDOW", client.on_closing)
    root.mainloop()
